In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


%matplotlib inline
from skimage.segmentation import mark_boundaries
import lime
from lime import lime_image
import cv2
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ReduceLROnPlateau , ModelCheckpoint
from collections import Counter

In [2]:
data_path = 'Data'
directory = 'Data/Train'
directory2 = 'Data/Test'

In [3]:
File=[]
for file in os.listdir(directory):
    File+=[file]
print(File)

['COVID19', 'NORMAL', 'PNEUMONIA']


In [4]:
dataset=[]
mapping={'COVID19':0, 'NORMAL':1, 'PNEUMONIA':2}
count=0

for file in os.listdir(directory):
    path=os.path.join(directory,file)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(224,224))
        image=img_to_array(image)
        image=image/255.0
        dataset.append([image,count])     
    count=count+1

In [5]:
testset=[]
mapping={'COVID19':0, 'NORMAL':1, 'PNEUMONIA':2 }
count=0

for file in os.listdir(directory2):
    path=os.path.join(directory2,file)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(224,224))
        image=img_to_array(image)
        image=image/255.0
        testset.append([image,count])         
    count=count+1

In [6]:
data,labels0=zip(*dataset)
test,testlabels0=zip(*testset)

In [7]:
labels1=to_categorical(labels0)
data=np.array(data)
labels=np.array(labels1)
print(data.shape)
print(labels.shape)

(5144, 224, 224, 3)
(5144, 3)


In [8]:
testlabels1=to_categorical(testlabels0)
test=np.array(test)
testlabels=np.array(testlabels1)
print(test.shape)
print(testlabels.shape)

(1288, 224, 224, 3)
(1288, 3)


In [9]:
data2=data.reshape(-1,224,224,3)
test2=test.reshape(-1,224,224,3)

In [10]:
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=22)

In [11]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

(4115, 224, 224, 3)
(1029, 224, 224, 3)
(4115, 3)
(1029, 3)


In [12]:
baseModel=DenseNet121(input_shape=(224,224,3),include_top=False,weights='imagenet',pooling='avg')

for layer in baseModel.layers:
  layer.trainable = False

headModel = baseModel.output
headModel = AveragePooling2D()(headModel)
headModel = Flatten()(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(3, activation='softmax')(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

ValueError: Input 0 of layer average_pooling2d is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 1024)

In [ ]:
model.summary()

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                    width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")